In [32]:
import pandas as pd
import requests
import json
import csv
import boto3

In [2]:
reader = pd.read_csv('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/New-SeasonToBrandMapping.csv')

In [11]:
FILENAME ='SeasonToBrandMapping-RohinisPlus.csv'
customerTypesList = ['BACKBOOK_NO_CINEMA','FRONTBOOK_OR_DEFAULT']
discoveryFeedRequestUrl = 'https://contentdata.vision.bt.com/cptmdss/V7.0/discoveryFeed'

brandRequestUrl = 'https://contentdata.vision.bt.com/cptmdss/V7.0/brandDetails'
brandRequestQueryParams = {
    'context': 'commendaroo-testing',
}

In [15]:
def callDiscoveryFeed(feedUrl, queryParams):
    request = requests.get(feedUrl, params=queryParams)    
    return request.json()["discoveryFeedResponse"]["items"]

def getSeasonsForBrand(brandGuid):
    brandRequestQueryParams["brandGuid"] = brandGuid
    request = requests.get(brandRequestUrl, params=brandRequestQueryParams)
    groups = request.json()["bdResponse"]["brandInfo"]["groups"]
    seasonGuids = []
    for season in groups:
        seasonGuids.append(season["guid"].encode("utf-8"))
    return seasonGuids

def getBrandListForFeedUrl(feedUrl, queryParams, brandGuidList, existingBrandGuids):
    queryParamsForCall = queryParams
    gotContent = True
    while gotContent == True:
        itemList = callDiscoveryFeed(feedUrl, queryParamsForCall)   ## Get items for this page (because response is paged)
        if (len(itemList) == 0):                                    ## If page had no items, we've reached the bottom of the available content
            gotContent = False                                      ##If the call returns nothing then we've got no data to work with so break
            continue
                
        for content in itemList:                                    ## If we have content, loop through each one
            if (content['type']) == 'BRAND':                        ## Select only brand contents
                if(content["guid"] in brandGuidList):               ## Check if we've found this brandGuid before on a previous call
                    continue                                        
                brandGuidList.append(content["guid"].encode("utf-8"))## If not pop it in the list

        queryParamsForCall['cursor'] = itemList[-1]["guid"]         ## Set us up for the next loop, telling it to use that last content as the cursor location
        queryParamsForCall['direction'] = 'FORWARD'

In [33]:
def read_data(file_name):
    
    s3 = boto3.client('s3') 
    obj = s3.get_object(Bucket= 'bt-data-science-playground', Key= file_name)
    
    return pd.read_csv(obj['Body'], delimiter=',', parse_dates=True)

existingSeasonToBrand_csv = read_data('bt-tv-recommendation-system/model_objects/SeasonToBrandMapping-RohinisPlus.csv')

In [27]:
existingSeasonToBrand = {}
for b in tmp['BRAND_GUID'].unique().tolist():
    existingSeasonToBrand[b] = existingSeasonToBrand_csv[existingSeasonToBrand_csv['BRAND_GUID'] == b]['SEASON_GUID'].unique().tolist()

In [7]:
existingBrandGuids = existingSeasonToBrand_csv['BRAND_GUID'].unique().tolist()
seasonToBrand = []
brandGuidList = []

In [8]:
for index, row in reader.iterrows():
    if(row['BRAND_GUID'] in existingSeasonToBrand):
        existingSeasonToBrand[row['BRAND_GUID']].append(row['SEASON_GUID'])
    else:
        existingSeasonToBrand[row['BRAND_GUID']] = [row['SEASON_GUID']]
    if(row['BRAND_GUID'] not in existingBrandGuids):
        existingBrandGuids.append(row['BRAND_GUID'])

In [18]:
tags = ['TV', 'TV Replay', 'Kids']
for tag in tags:
    for customerType in customerTypesList:
        queryParams = {
            'byTags': '{schedulerChannel, ' + tag + '}',
            'context': 'commendaroo-testing',
        }
        queryParams['customerType'] = customerType
        getBrandListForFeedUrl(discoveryFeedRequestUrl, queryParams, brandGuidList, existingBrandGuids)
        print( tag + 'BrandList + ' + customerType + ' full brand list len : ' + str(len(brandGuidList)))


TVBrandList + BACKBOOK_NO_CINEMA full brand list len : 391
TVBrandList + FRONTBOOK_OR_DEFAULT full brand list len : 781
TV ReplayBrandList + BACKBOOK_NO_CINEMA full brand list len : 1050
TV ReplayBrandList + FRONTBOOK_OR_DEFAULT full brand list len : 1062
KidsBrandList + BACKBOOK_NO_CINEMA full brand list len : 1164
KidsBrandList + FRONTBOOK_OR_DEFAULT full brand list len : 1266


In [20]:
############################
####### Get Seasons ########
############################
for brandGuid in brandGuidList:
    ## Check if this brandGuid exists already, if not add it, if it does check the seasonGuid
    if (brandGuid not in existingSeasonToBrand):
        for seasonGuid in getSeasonsForBrand(brandGuid):
            ## Add these to our dictionary
            seasonToBrand.append({
                "SEASON_GUID" : seasonGuid,
                "BRAND_GUID" : brandGuid
            })
    else:
        for seasonGuid in getSeasonsForBrand(brandGuid):
            ## Check if the seasonguid is already recorded for this brand
            if (seasonGuid not in existingSeasonToBrand[brandGuid]):
                ## Add these to our dictionary
                seasonToBrand.append({
                    "SEASON_GUID" : seasonGuid,
                    "BRAND_GUID" : brandGuid
                })

3055
[{'SEASON_GUID': b'movida_10018983', 'BRAND_GUID': b'movida_21824'}, {'SEASON_GUID': b'movida_10088927', 'BRAND_GUID': b'movida_21824'}, {'SEASON_GUID': b'movida_10019409', 'BRAND_GUID': b'movida_14414'}, {'SEASON_GUID': b'movida_10003143', 'BRAND_GUID': b'movida_14414'}, {'SEASON_GUID': b'movida_56415', 'BRAND_GUID': b'movida_10827'}, {'SEASON_GUID': b'movida_56419', 'BRAND_GUID': b'movida_10827'}, {'SEASON_GUID': b'movida_56417', 'BRAND_GUID': b'movida_10827'}, {'SEASON_GUID': b'movida_56421', 'BRAND_GUID': b'movida_10827'}, {'SEASON_GUID': b'movida_61024', 'BRAND_GUID': b'movida_14110'}, {'SEASON_GUID': b'movida_10030042', 'BRAND_GUID': b'movida_21568'}, {'SEASON_GUID': b'movida_10018551', 'BRAND_GUID': b'movida_21568'}, {'SEASON_GUID': b'movida_10079363', 'BRAND_GUID': b'movida_21568'}, {'SEASON_GUID': b'movida_56317', 'BRAND_GUID': b'movida_10771'}, {'SEASON_GUID': b'movida_56316', 'BRAND_GUID': b'movida_10771'}, {'SEASON_GUID': b'movida_56315', 'BRAND_GUID': b'movida_10771'}

In [22]:
with open(FILENAME, 'w') as csvfile:
    fieldnames = ['SEASON_GUID', 'BRAND_GUID']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for item in seasonToBrand:
        writer.writerow(item)

TypeError: a bytes-like object is required, not 'str'